In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import gc
gc.enable()

In [3]:
PATH = '../finanz_vs_news/'

In [4]:
df_reu = pd.read_csv(PATH+'reuters_ALL.csv.gz', parse_dates=['Time']).set_index('Time')

In [6]:
df_reu.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5933715 entries, 2007-01-02 04:47:49 to 2017-12-30 05:05:43
Data columns (total 1 columns):
Short_Msg    object
dtypes: object(1)
memory usage: 734.5 MB


In [7]:
#temporary measure to remove suspicious dates
#df_reu = df_reu.loc['2008',:]

### Read in company ticker names

In [8]:
companies = pd.read_pickle('TickerDic.pickle')

In [9]:
companies.head()

,Ticker,Name,Category Name,RedName
0,AFL,Aflac Incorporated,Accident & Health Insurance,Aflac
1,AIZ,"Assurant, Inc.",Accident & Health Insurance,Assurant
23,UNM,Unum Group,Accident & Health Insurance,Unum
52,IPG,"The Interpublic Group of Companies, Inc.",Advertising Agencies,Interpublic
57,OMC,Omnicom Group Inc.,Advertising Agencies,Omnicom


In [10]:
company_names = companies['RedName'].tolist()

In [11]:
companies.query('RedName == "Morgan Stanley"')

,Ticker,Name,Category Name,RedName
12065,MS,Morgan Stanley,Investment Brokerage - National,Morgan Stanley


### Red in neg/pos dictionaries

In [12]:
pos_words = pd.read_pickle('DicPositiveWords.pickle')
neg_words = pd.read_pickle('DicNegativeWords.pickle')

### Do bag of words

In [13]:
import helpers

import importlib
importlib.reload(helpers)

<module 'helpers' from '/media/mlisovyi/shared_space/dfine2018/dfine_hack2018/helpers.py'>

In [14]:
media_attention = helpers.count_sensitive_words(df_reu.copy(deep=True), 
                                        ["Morgan Stanley", "Goldman Sachs", "BlackRock"]+[{'positive': pos_words.values.tolist()},
                                                       {'negative': neg_words.values.tolist()}
                                                      ], 
                                        companies_neg_pos=["Morgan Stanley", "Goldman Sachs", "BlackRock"])

In [15]:
media_attention = helpers.fe(media_attention)

In [16]:
media_attention.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4556 entries, 2005-07-22 to 2018-01-10
Freq: D
Data columns (total 23 columns):
Morgan_Stanley_count             4556 non-null int64
Goldman_Sachs_count              4556 non-null int64
BlackRock_count                  4556 non-null int64
positive_count                   4556 non-null int64
negative_count                   4556 non-null int64
Morgan_Stanley_positive_count    4556 non-null int64
Morgan_Stanley_negative_count    4556 non-null int64
Goldman_Sachs_positive_count     4556 non-null int64
Goldman_Sachs_negative_count     4556 non-null int64
BlackRock_positive_count         4556 non-null int64
BlackRock_negative_count         4556 non-null int64
total_msg_count                  4556 non-null int64
Morgan_Stanley_frac              3439 non-null float64
Goldman_Sachs_frac               3439 non-null float64
BlackRock_frac                   3439 non-null float64
positive_frac                    3439 non-null float64
negative_fr

In [17]:
media_attention.sum().sort_values(ascending=False)

total_msg_count                  5.933715e+06
negative_count                   2.028449e+06
positive_count                   1.042191e+06
Morgan_Stanley_count             1.563300e+04
BlackRock_negative_count         1.048500e+04
Goldman_Sachs_count              9.898000e+03
BlackRock_count                  8.324000e+03
Morgan_Stanley_negative_count    5.199000e+03
Goldman_Sachs_negative_count     3.644000e+03
negative_frac                    1.376888e+03
Morgan_Stanley_positive_count    9.550000e+02
BlackRock_positive_count         7.150000e+02
positive_frac                    5.622091e+02
Goldman_Sachs_positive_count     2.770000e+02
Morgan_Stanley_frac              5.606175e+00
BlackRock_negative_frac          3.713565e+00
Goldman_Sachs_frac               3.420271e+00
BlackRock_frac                   2.880341e+00
Morgan_Stanley_negative_frac     1.900912e+00
Goldman_Sachs_negative_frac      1.257423e+00
Morgan_Stanley_positive_frac     3.892451e-01
BlackRock_positive_frac          2

In [18]:
print('Numbers of negative and positive mentions of Morgan Stanley are: {} and {}'.format(media_attention['Morgan_Stanley_negative_count'].sum(),
                                                                                   media_attention['Morgan_Stanley_positive_count'].sum())
     )

Numbers of negative and positive mentions of Morgan Stanley are: 5199 and 955


#### Save the pre-processed media dataset

In [19]:
media_attention.to_csv('MediaAttention_Mini.csv')